In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy import stats

def initialize_graph(n, epsilon, w0):
    W = np.full((n, n), epsilon)
    np.fill_diagonal(W, w0)
    return W

def update_weights(W, alpha, beta, p, theta):
    n = W.shape[0]
    total_weight = W.sum()
    
    # Edge selection
    probs = W.flatten() / total_weight
    selected_edge = np.random.choice(n*n, p=probs)
    i, j = selected_edge // n, selected_edge % n
    
    # Weight update
    w_i = W[i, :].sum()
    w_j = W[j, :].sum()
    delta_w = alpha * W[i, j] * (w_i + w_j) / (2 * total_weight)
    W[i, j] += delta_w
    W[j, i] = W[i, j]  # maintain symmetry
    
    # Normalization
    W *= total_weight / W.sum()
    
    # Noise injection
    if np.random.random() < p:
        i, j = np.random.randint(0, n, 2)
        W[i, j] += beta * epsilon
        W[j, i] = W[i, j]
    
    # Thresholding
    W[W < theta] = theta
    
    return W

def run_simulation(n, T, epsilon, w0, alpha, beta, p, theta):
    W = initialize_graph(n, epsilon, w0)
    weights_history = []
    clustering_coeffs = []
    
    for t in range(T):
        W = update_weights(W, alpha, beta, p, theta)
        weights_history.append(W.flatten())
        
        # Calculate clustering coefficient
        G = nx.from_numpy_array(W)
        clustering_coeffs.append(nx.average_clustering(G, weight='weight'))
    
    return W, weights_history, clustering_coeffs

def plot_results(W, weights_history, clustering_coeffs):
    # Plot 1: Histogram of edge weights
    plt.figure(figsize=(10, 8))
    plt.hist(W.flatten(), bins=50, density=True)
    plt.title('Distribution of Edge Weights')
    plt.xlabel('Weight')
    plt.ylabel('Density')
    plt.yscale('log')
    plt.xscale('log')
    plt.grid(True)
    plt.savefig('weight_distribution.png')
    plt.close()

    # Plot 2: Evolution of clustering coefficient
    plt.figure(figsize=(10, 8))
    plt.plot(clustering_coeffs)
    plt.title('Evolution of Average Clustering Coefficient')
    plt.xlabel('Time Step')
    plt.ylabel('Average Clustering Coefficient')
    plt.grid(True)
    plt.savefig('clustering_coefficient.png')
    plt.close()

    # Plot 3: Graph visualization
    G = nx.from_numpy_array(W)
    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, node_size=30, with_labels=False)
    edge_weights = [W[u][v] * 10000 for u, v in G.edges()]
    nx.draw_networkx_edges(G, pos, width=edge_weights, alpha=0.2)
    plt.title('Graph Visualization (edge thickness represents weight)')
    plt.axis('off')
    plt.savefig('graph_visualization.png')
    plt.close()

    # Plot 4: Evolution of specific edges
    num_edges_to_track = 5
    tracked_edges = np.random.choice(len(W.flatten()), num_edges_to_track, replace=False)
    plt.figure(figsize=(10, 8))
    for edge in tracked_edges:
        plt.plot([w[edge] for w in weights_history], label=f'Edge {edge}')
    plt.title('Evolution of Specific Edge Weights')
    plt.xlabel('Time Step')
    plt.ylabel('Weight')
    plt.legend()
    plt.grid(True)
    plt.savefig('edge_weight_evolution.png')
    plt.close()

# Run simulation
n = 100  # number of nodes
T = 10000  # number of time steps
epsilon = 0.01
w0 = 0.1
alpha = 0.1
beta = 0.5
p = 0.01
theta = 0.001

final_weights, weights_history, clustering_coeffs = run_simulation(n, T, epsilon, w0, alpha, beta, p, theta)

# Plot results
plot_results(final_weights, weights_history, clustering_coeffs)

print("Simulation complete. Check the generated PNG files for visualizations.")

KeyboardInterrupt: 